In [ ]:
import datetime
import pandas as pd
import plotly.graph_objects as go

# 1) Подготовим примерные данные
epics = pd.DataFrame([
    dict(name="Epic 1", start="2025-07-01", end="2025-07-20"),
    dict(name="Epic 2", start="2025-08-10", end="2025-09-05"),
    dict(name="Epic 3", start="2025-07-15", end="2025-09-15"),
])
epics["Start"] = pd.to_datetime(epics["start"])
epics["End"]   = pd.to_datetime(epics["end"])
epics["Duration"] = (epics["End"] - epics["Start"]).dt.days

# 2) Определим месяцы и недели для фона
months = [
    ("2025-07-01", "2025-07-31", "#A8E6CF", "Июль"),
    ("2025-08-01", "2025-08-31", "#FFD3B6", "Август"),
    ("2025-09-01", "2025-09-30", "#A8E6CF", "Сентябрь"),
]
# генерим по 1-ой секунде каждого дня
weeks = pd.date_range("2025-07-01", "2025-09-30", freq="W-MON")

fig = go.Figure()

# 3) Добавим месяцовые фоны
for start, end, color, label in months:
    fig.add_shape(type="rect",
                  x0=start, x1=end,
                  y0=-0.5, y1=len(epics)-0.5,
                  fillcolor=color, line_width=0, layer="below")
    # и подпись месяца над графиком
    mid = (pd.to_datetime(start) + (pd.to_datetime(end)-pd.to_datetime(start))/2)
    fig.add_annotation(x=mid, y=1.02, xref="x", yref="paper",
                       text=label, showarrow=False, font_size=16)

# 4) Добавим вертикальные линии-демо (пример)
demo_dates = ["2025-07-15", "2025-08-15", "2025-09-15"]
for d in demo_dates:
    dt = pd.to_datetime(d)
    fig.add_shape(
        type="rect",
        x0=dt - pd.Timedelta(days=0.3),
        x1=dt + pd.Timedelta(days=0.3),
        y0=-0.5,
        y1=len(epics)-0.5,        # или ваша длина списка задач
        xref="x",
        yref="y",
        fillcolor="#FFECB3",
        line_width=0,
        opacity=0.3,
        layer="below"
    )


# 5) Полосы эпиков
for i, row in epics.iterrows():
    fig.add_trace(go.Bar(
        x=[row["Duration"]],
        y=[row["name"]],
        base=[row["Start"]],
        orientation="h",
        marker_color="#6A1B9A",
        hovertemplate=(
            "<b>%{y}</b><br>"
            "Начало: %{base|%d.%m.%Y}<br>"
            "Конец: %{x+base|%d.%m.%Y}<extra></extra>"
        )
    ))

# 6) Настроим ось X и Y
fig.update_layout(
    barmode="overlay",
    xaxis=dict(
        tickformat="%d.%m",
        tickmode="array",
        tickvals=weeks,
        ticktext=[f"Неделя {w.isocalendar()[1]}" for w in weeks],
        range=["2025-07-01", "2025-09-30"]
    ),
    yaxis=dict(autorange="reversed"),  # чтобы Epic 1 был сверху
    margin=dict(t=80, b=80),
    height=400,
)

fig.show()
